In [7]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize

## Group Assignment
### Team Number: 11
### Team Member Names: Akram, Annie, Jester
### Team Strategy Chosen: Market Beat

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


## **General Strategy for the Project**:

#### Initialization
0) Define and initialize necessary global variables

#### Part #1: Data Filtering and Cleaning
1) Filter out all valid US and CAD Stocks from the provided CSV file (Storing US tickers in another list for currency conversion later on)
2) Download and store closing price, options and volume data in a dictionary using yfinance.
3) Filter out tickers within date range based on given minimum monthly average volume.

#### Part #2: Portfolio Construction
4) Rank stocks based on Standard Deviation of percentage change in returns in descending order.
5) Rank stocks based on PCR values using options data in descending order.
6) Score the stocks based on the two ranks, and create a new ranking based on the scoring.
7) We select stocks based on the ranking and calculate weights that would maximize the portfolio sharpe ratio while beta is within pre-defined constraints.
8) Run sharpe ratio calculation function on portfolio from size 12(min) to 24(max), so that the final portfolio (with weightings) is picked based on highest output sharpe ratio.

#### Part #3: Evaluation and Proof
9) Given the chosen portfolio of xx stocks, graph the change in portfolio standard deviation as other stocks are added.
10) Beta of portfolio compared to the S&P 500.
11) Calculate the Beta between our portfolio and an equally weighted portfolio including all valid stocks.
12) Beta between our portfolio with varied weight versus when the portfolio is equally weighted.
13) Sharpe ratio between our portfolio with varied weight versus when it is equally weighted.
14) Graphically compare sharpe ratios amongst our portfolio of xx stocks and the portfolios of varying 12-24 stocks.

#### Part #4: Final Output
15) Creating the final portfolio dataframe and CSV.

## Initializing Variables

In [8]:
def get_tickers():
    tickers = pd.read_csv('Tickers.csv')
    ticker_lst = [tickers.columns[0]] + (list(tickers[tickers.columns[0]]))
    return ticker_lst

In [9]:
# Important Constants: 
amount = 1_000_000 # Initial investment amount of $1,000,000
group = 11

# Define constants
min_avg_volume = 100000
min_trading_days = 18
start_date, end_date = '2022-09-30', '2024-09-30'
min_stocks, max_stocks = 12, 24

# Reading in CSV file: 
tickers = pd.read_csv('Tickers.csv')
ticker_lst = get_tickers()

# Initializing variable to store the tickers we will use in our portfolio
columns = ['Ticker', 'Price', 'Currency', 'Shares', 'Value', 'Weight']
Portfolio_Final = pd.DataFrame(columns=columns)
exchange_rate = yf.Ticker('CAD=X').fast_info['last_price']
print(f'The current exchange rate for the latest available day:\nUSD -> CAD: ${np.round(exchange_rate, 4)}')

The current exchange rate for the latest available day:
USD -> CAD: $1.3987


#### We must filter the tickers csv as follows:
- Must be listed on yfinance
- The currency is listed as USD or CAD 
- 100,000+ average monthly volume trades
- More than 18 trades per month
- Sufficient data

In [10]:
# Drop tickers with less than 18 days of trading data
def drop_short_trading_months(df):
    """
    Drops months with less than 18 trading days from a yfinance history DataFrame.
    Parameters:
        df (pd.DataFrame): A yfinance DataFrame with a DateTimeIndex and stock data.
    Returns:
        pd.DataFrame: Filtered DataFrame with only months having >= 18 trading days.
    """
    # Ensure the index is datetime for proper grouping
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame index must be a DateTimeIndex.")
    # Group by month and count trading days
    trading_days_per_month = df.groupby(df.index.to_period('M')).size()
    # Identify months with at least 18 trading days
    valid_months = trading_days_per_month[trading_days_per_month >= 18].index
    # Filter the original DataFrame to include only valid months
    filtered_df = df[df.index.to_period('M').isin(valid_months)]
    return filtered_df

In [11]:
# Filtering valid stocks by inputting a list of strings for each ticker. 
def filter_stocks(ticker_lst):
    valid_tickers, invalid_tickers = {}, []
    # Loop through all tickers to check if they are valid
    for ticker in ticker_lst:
        stock = yf.Ticker(ticker)
        try:
            info = stock.fast_info # Get basic stock info

            hist = stock.history(start=start_date, end=end_date) # Get stock history
            hist.index = hist.index.strftime('%Y-%m-%d')

            avg_volume = hist.loc[((hist.index >= start_date) & (hist.index <= end_date))]['Volume'].mean() # Calculate average volume in specified date range.
            currency = info.get("currency")
            if ((hist.empty is not None) and # filter for stocks delisted on yfinance
                ( currency == "USD" or currency == "CAD") and # filter for stocks that are not USD
                (avg_volume >= min_avg_volume)): # Filter by volume greater than 100,000
                if currency == "CAD":
                    valid_tickers[ticker] = hist['Close'] # Store the close prices of the stock as a Series
                elif currency == "USD":
                    valid_tickers[ticker] = hist['Close'] * exchange_rate # Convert USD to CAD
            else:
                invalid_tickers.append(ticker)
        except:
            invalid_tickers.append(ticker)
    return [valid_tickers, invalid_tickers]
    # valid_tickers is a dictionary of Series where the key is the name of the ticker. 
    # invalid_tickers is a list of ticker strings which were removed in the filtering process. 

In [13]:
def calculate_std(data):
    data.index = pd.to_datetime(data.index)
    
    # Calculate daily percentage returns
    returns = data.pct_change().dropna()

    # Calculate standard deviation of returns
    std = pd.DataFrame(returns.std(), columns=['Standard Deviation'])

    # Sort by standard deviation
    std_sorted = std.sort_values(by='Standard Deviation', ascending=False)

    # Add Rank column
    std_sorted['Rank'] = range(len(std_sorted))

    # Add Score column
    highest_std_value = std_sorted['Standard Deviation'].iloc[0]
    std_sorted['Score'] = (std_sorted['Standard Deviation'] / highest_std_value) * 100

    return std_sorted

In [14]:
# Loading data into variables
stock_filter = filter_stocks(ticker_lst)
ticker_data = stock_filter[0]
ticker_lst = list(ticker_data.keys()) # Reassign original ticker list
data = pd.DataFrame()
for ticker in ticker_data:
    data[ticker] = ticker_data[ticker]

# returns = data.pct_change()
# returns.drop(index=returns.index[0], inplace = True)

data.head()

$AGN: possibly delisted; no timezone found
$AIG: possibly delisted; no price data found  (1d 2022-09-30 -> 2024-09-30)
$CELG: possibly delisted; no timezone found
$MON: possibly delisted; no timezone found
$PG: possibly delisted; no price data found  (1d 2022-09-30 -> 2024-09-30)
Failed to get ticker 'RTN' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)
$RTN: possibly delisted; no timezone found


,AAPL,ABBV,ABT,ACN,AMZN,AXP,BA,BAC,BB.TO,BIIB,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-30,190.973304,172.315307,129.477849,347.111309,158.054231,183.050859,169.355811,39.937660,6.51,373.455573,...,150.759942,113.588989,37.189999,24.323669,76.140358,202.476400,685.713400,259.936349,205.385741,51.012701
2022-10-03,196.846217,177.592216,133.171083,357.350688,162.082512,189.984286,176.307401,41.114633,6.53,369.679039,...,156.417786,114.575356,37.799999,25.033068,77.623268,209.095686,699.928909,267.194590,207.356454,52.594194
2022-10-04,201.890054,182.304251,137.934841,370.058724,169.369790,197.365461,186.741766,42.820584,6.74,374.085010,...,163.289918,117.543648,42.610001,25.316833,79.079201,216.041983,710.329169,274.025894,214.730714,53.935300
2022-10-05,202.304571,184.024680,138.336275,370.099147,169.173972,195.886503,184.783581,42.212260,6.67,366.028427,...,166.652576,117.041313,41.959999,25.104010,78.620857,219.508601,715.624572,267.674906,213.510172,53.391272
2022-10-06,200.964190,180.121562,137.091808,363.529206,168.264819,193.186405,184.909459,41.603934,6.64,366.825702,...,167.172984,113.762512,41.320000,24.421211,75.751678,217.860289,704.843987,264.459405,210.547774,51.974260


In [ ]:
# Function to get the total volume for a call or put of a given stock.
# ticker: yfinance Ticker class
# put: Boolean for if you want to calculate put volume. Else, put False for call volume. 
def get_options_vol(ticker, put):
    exps = ticker.options # Expiration dates of available options
    optdata = pd.DataFrame() # Data storage
    for exp in exps:
        chain = pd.DataFrame()
        if put: chain = ticker.option_chain(exp).puts['volume'] # Gets the desired columns
        else: chain = ticker.option_chain(exp).calls['volume'] # If put options are desired then use this data.
        optdata = pd.concat([optdata, chain]) # Add the calls/puts to the main dataframe. 
    return optdata.sum()['volume'] # output total volue of put/call options

# Function to calculate the PCR for each stock. 
def PCR_calc(tickers):
    pcrdata = pd.DataFrame(columns=['Ticker', 'Put Volume', 'Call Volume', 'PCR'])
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        try: 
            # Get the volume for Put and Call options:
            call_options = get_options_vol(stock, False)
            put_options = get_options_vol(stock, True)
            # Calculate PCR Ratio:
            pcr = call_options / put_options # Order reversed from the formula for sake of ranking
            #print(f"Ticker: {ticker}, PCR: {pcr}")  # Debugging
            pcrdata.loc[len(pcrdata)] = [ticker, put_options, call_options, pcr]
        except Exception as e:
            print(f"Options Data Not Found {ticker}: {e} not found")  # Debugging (output error)
            pass
    return pcrdata

In [16]:
std = calculate_std(data)
std

/var/folders/9g/h_n63rvn21s1k0y3xvf0n28h0000gn/T/ipykernel_80529/2355861676.py:5: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change().dropna()


,Standard Deviation,Rank,Score
SHOP.TO,0.036119,0,100.000000
BB.TO,0.034555,1,95.669034
PYPL,0.023802,2,65.898661
QCOM,0.023472,3,64.984553
AMZN,0.021319,4,59.023566
USB,0.021262,5,58.864445
BA,0.020383,6,56.431218
LLY,0.018070,7,50.027400
CAT,0.017667,8,48.911560
TXN,0.016711,9,46.266554


In [17]:
# Load the PCR values for each of the valid stocks into a variable
options_data = PCR_calc(ticker_lst)
options_data = options_data.sort_values(by='PCR', ascending=False)
options_data['Rank'] = [i for i in range(len(options_data))]
highest_pcr = options_data['PCR'].iloc[0]
options_data['Score'] = (options_data['PCR'] / highest_pcr) * 100
options_data.set_index('Ticker', inplace=True)


# Display the table of rankings based off PCR. 
# The rankings are based off the stocks with the greatest sentiment for if they will go up or not
# The tickers at the top of the list have a high call rate (meaning the price will go up)
pcr = options_data
pcr

Error processing BB.TO: 'volume'
Error processing RY.TO: 'volume'
Error processing SHOP.TO: 'volume'
Error processing T.TO: 'volume'
Error processing TD.TO: 'volume'


,Put Volume,Call Volume,PCR,Rank,Score
Ticker,,,,,
KO,10326.0,30712.0,2.974240,0,100.000000
PYPL,9487.0,25496.0,2.687467,1,90.358117
PEP,4726.0,12210.0,2.583580,2,86.865229
AMZN,120536.0,258077.0,2.141078,3,71.987410
MRK,8025.0,15846.0,1.974579,4,66.389383
BA,22965.0,42704.0,1.859525,5,62.521031
LLY,26516.0,42699.0,1.610311,6,54.141928
UPS,9503.0,14485.0,1.524255,7,51.248575
PM,3712.0,5657.0,1.523976,8,51.239187


In [18]:
def calculate_scoreboard(std, pcr):
    """
    Merges two DataFrames (std and pcr) on their index (assumed to be ticker names),
    calculates the average of their 'Score' columns, and sorts the result by 'Average Score'.
    """

    merged = std[['Score']].merge(pcr[['Score']], left_index=True, right_index=True, suffixes=('_std', '_pcr'), how='outer')

    # interesting case here, we need to consider stocks with nan from PCR strategy
    merged['Score_std'].fillna(merged['Score_pcr'], inplace=True)
    merged['Score_pcr'].fillna(merged['Score_std'], inplace=True)
    
    # Calculate the average score
    merged['Average Score'] = merged[['Score_std', 'Score_pcr']].mean(axis=1)
    
    # Sort the DataFrame by 'Average Score' in descending order
    merged_sorted = merged.sort_values(by='Average Score', ascending=False)
    
    return merged_sorted

calculate_scoreboard(std, pcr)

/var/folders/9g/h_n63rvn21s1k0y3xvf0n28h0000gn/T/ipykernel_80529/1719902720.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged['Score_std'].fillna(merged['Score_pcr'], inplace=True)
/var/folders/9g/h_n63rvn21s1k0y3xvf0n28h0000gn/T/ipykernel_80529/1719902720.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are sett

,Score_std,Score_pcr,Average Score
SHOP.TO,100.000000,100.000000,100.000000
BB.TO,95.669034,95.669034,95.669034
PYPL,65.898661,90.358117,78.128389
AMZN,59.023566,71.987410,65.505488
KO,24.033255,100.000000,62.016628
BA,56.431218,62.521031,59.476124
PEP,28.227776,86.865229,57.546502
QCOM,64.984553,48.829463,56.907008
LLY,50.027400,54.141928,52.084664
MRK,34.979984,66.389383,50.684683


## sharpe ratio optimization to determine weights

In [1]:
# sharpe ratio optimization

def optimal_sharpe(tickers, start_date, end_date, risk_free_rate, investment):
    # download data
    data = yf.download(tickers, start=start_date, end=end_date)['Close'].dropna() 

    def neg_sharpe(weights):

        # determining number of shares of each stock that can be bought
        shares = []
        for i in range(len(tickers)):
            allocation = investment * weights[i]  # investment allocated to this stock
            price_per_share = data.iloc[0][tickers[i].upper()]
            
            # clculate fees
            flat_fee = 3.95
            per_share_fee = allocation/price_per_share/(1000+1/price_per_share) 
            
            # choose the smaller of the two fees
            trading_fee = min(flat_fee, per_share_fee)
            
            # calculate the number of shares after deducting the fee
            effective_investment = allocation - trading_fee
            shares.append(effective_investment / price_per_share)
        
        # forming the portfolio
        portfolio = data*shares
        portfolio['total'] = portfolio.sum(axis=1)
        portfolio['daily return'] = portfolio['total'].pct_change(1)

        # calculating sharpe ratio
        er = portfolio['daily return'].mean()
        std = portfolio['daily return'].std()
        sharpe_ratio = (er-risk_free_rate)/std
        
        sharpe_ratio = sharpe_ratio*(252**0.5) # annualizing sharpe ratio by trading days

        return -sharpe_ratio #make sharpe ratio negative for minimize function

    # constraints
    def check_sum(weights): 
        return np.sum(weights)-1 #returns 0 if weights sum up to 1
    constraints = {'type': 'eq', 'fun': check_sum}

    min_weight = 1/(2*len(tickers))
    max_weight = 0.4

    bounds = [(min_weight, max_weight)]*len(tickers)

    # initial guess
    init_guess = [1.0/len(tickers)]*len(tickers)

    results = minimize(neg_sharpe, init_guess, method="SLSQP", bounds=bounds, constraints=constraints)

    return results

tickers = sorted(['aapl','adbe','amd','avgo','crm', 'csco', 'fi','fis', 'ibm', 'intc', 'lrcx', 'msft', 'mu', 'nvda',  'now', 'orcl', 'qcom', 'txn'])

optimal = optimal_sharpe(tickers, "2017-01-01", "2018-01-01", 0, 100)
print(optimal.x)

NameError: name 'yf' is not defined

In [ ]:
# Code to output final dataframe to a CSV file called Stocks_Group_XX.csv
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']]
Stocks_Final.to_csv(f'Stocks_Group_{group}.csv', index=False)

In [20]:
Portfolio_Final

,Ticker,Price,Currency,Shares,Value,Weight


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

---
<p style="color: #004dd3">
Akram Jamil
</p>

<p style="color: #2C8CA9">
Jester Yang
</p>

<p style="color: #3cc19d;">
Annie Wong
</p>

---